### Tesi M.N.A

In [1]:
%load_ext autoreload
%autoreload 2
from Tools import *

#### loading data

In [ ]:
rock_cropout, mean_elev, stazioni, unconsolidated, rocks, coastline, clusters, cluster_data = load_data_from_files()

C:\Users\mario\Desktop\Code\tesi\Tools.py:372: DtypeWarning: Columns (7,11,12,27) have mixed types.Specify dtype option on import or set low_memory=False.
  rocks = csv_to_gpd(f"{root_path}rocks")


### ADD STATION CODE AS ID

In [ ]:
def get_data():
    filenames = glob(f'data{os.sep}step_2_data_input{os.sep}clima{os.sep}amrc{os.sep}*{os.sep}*.dat')
    names = ['station', 'timestamp', 'latitude', 'longitude', 'elevation', 'temperature', 'pressure',
                   'wind_speed', 'wind_direction', 'relative_humidity', 'vertical_temperature_difference']
    count = 0
    skipped = 0
    rows = []
    for file in tqdm(filenames[:]):
        with open(file) as f:
            lines = f.readlines()
            infos = [x for x in lines[0].split(' ') if len(x) > 0]
            station = lines[0].split(':')[1].strip('\n').replace(' ','')
            year = '19' + str(infos[1])
            if int(infos[1]) <= 21:
                year = '20' + str(infos[1])
            month = infos[0]
            base_dt = pd.Timestamp(year+'/'+month)
            line = lines[1].lower()
            try: latitude = line.split(':')[1].split('l')[0].lower().replace(' ','')
            except:  latitude = None
            try: longitude = line.split(':')[2].split('el')[0].lower().replace(' ','')
            except: longitude = None
            try: elevation = line.split(':')[3].replace(' ','').strip('m\n')
            except: elevation = None
            
            lines = lines[2:]
            for i in range(len(lines)-1):
                try:
                    line = lines[i]
                    infos = [x for x in line.split(' ') if len(x) > 0]
                    temperature = float(infos[0])
                    pressure = float(infos[1])
                    wind_speed = float(infos[2])
                    wind_direction = float(infos[3].strip('\n'))
                    relative_humidity = None
                    vertical_temperature_difference = None
                    if len(infos) == 6:
                        relative_humidity = float(infos[4])
                        vertical_temperature_difference = float(infos[5].strip('\n'))
                    timestamp = base_dt + timedelta(hours=3*(i-2))
                    row = [station, timestamp, latitude, longitude, elevation, temperature, pressure,
                       wind_speed, wind_direction, relative_humidity, vertical_temperature_difference]
                    rows.append(row)
                    count += 1
                except Exception as e:
                    skipped += 1
    print("skipped : ", skipped)
    amrc =  pd.DataFrame(rows, columns=names).replace(444.0, np.nan).replace('', np.nan)
    def long_cast(long):
        if type(long) == str:
            if long[-1] == 'e':
                return float(long[:-1])
            else:
                return -float(long[:-1])
        else:
            return None
    amrc.latitude = amrc.latitude.apply(lambda x: -float(x[:-1]) if type(x) == str else None)
    amrc.longitude = amrc.longitude.apply(lambda x: long_cast(x))
    amrc.timestamp = pd.to_datetime(amrc.timestamp)
    amrc['year'] = amrc.timestamp.dt.year
    amrc['month'] = amrc.timestamp.dt.month
    amrc['week'] = amrc.timestamp.dt.isocalendar().week
    return amrc

###### Dome Fuji (8904) is an exception to this, where the relative humidity column is instead lower temperature (C).

In [ ]:
amrc = get_data()
amrc

In [ ]:
px.line(x="year", y="number_of_records", data_frame=amrc.groupby("year").station.count().reset_index().rename(columns={"station" : "number_of_records"}))

In [ ]:
amrc = amrc[(amrc.year >= 2010) & (amrc.year <= 2020)]

In [ ]:
viz1 = amrc.groupby('station').agg({'latitude' : 'mean', 'longitude' : 'mean'}).reset_index()
basemap = get_basemap()
points_x, points_y = basemap(viz1.longitude, viz1.latitude)
viz1_proj = pd.DataFrame({'x': points_x, 'y': points_y})

fig, ax = plt.subplots(figsize=(10, 10))
basemap = viz_init(ax)
basemap.scatter(viz1_proj.x, viz1_proj.y, color='red', zorder=5, s=15, ax=ax)
plt.plot()

### Extract unique of amrc stations

In [ ]:
amrc_stat = amrc.groupby("station").agg({'latitude' : 'mean', 'longitude' : 'mean'}).reset_index() #Mean of coords, tipically coords can differ by a little amount

amrc_stat = gpd.GeoDataFrame({'station': amrc_stat.station.values}, geometry=gpd.points_from_xy(amrc_stat.longitude.values, amrc_stat.latitude.values)).set_crs('epsg:4326').to_crs('epsg:3031')

### Integrate AWS stations with clusters info

In [ ]:
clusters['AWS_IDs'] = clusters['geometry'].apply(lambda x: [idx for idx, geom in enumerate(amrc_stat.geometry.values) if x.contains(geom)])
clusters

## Example Query on weather data

In [ ]:
def filter_AWS_data(amrc, amrc_stat, from_date='2010-01-01', to_date='2021-06-01', variables=None, stations=None):
    if stations == None:
        stations = amrc_stat.index.values
    stations = amrc_stat[amrc_stat.index.isin(stations)].station
    if variables == None:
        variables = amrc.columns.values
    return amrc[(amrc.station.isin(stations)) & (amrc.timestamp > from_date) & (amrc.timestamp < to_date)][variables]

In [ ]:
clst_0_2020 = filter_AWS_data(amrc, amrc_stat, 
                from_date = '2019-01-01', 
                to_date   = '2020-01-01',
                variables = ['station', 'timestamp', 'wind_speed', 'wind_direction'],
                stations  = clusters.iloc[0].AWS_IDs)
clst_0_2020.head()

In [ ]:
for idx, row in clusters.iterrows():
    clust_data = filter_AWS_data(amrc, amrc_stat, stations=row["AWS_IDs"])
    if len(clust_data) > 0:
        print(f"Cluster number: {idx}\t median wind speed: {np.median(clust_data.wind_speed.dropna().values)}")
    else:
        print(f"Cluster number; {idx} has no AWS nearby")

### Wind Info

In [ ]:
wind_info = amrc.groupby(["latitude", "longitude", "year", "month", "week"]).agg({'wind_speed' : 'mean', 'wind_direction' : 'mean', 'station' : 'first'}).reset_index().fillna(0)
wind_info = gpd.GeoDataFrame(wind_info, geometry=gpd.points_from_xy(wind_info.longitude, wind_info.latitude), crs="EPSG:4326")
wind_info['strong_wind'] = wind_info.wind_speed.apply(lambda x: x>6)
wind_info['u'] = wind_info.apply(lambda x: x.wind_speed*(np.cos(x.wind_direction * np.pi/180)), axis=1)
wind_info['v'] = wind_info.apply(lambda x: x.wind_speed*(np.sin(x.wind_direction * np.pi/180)), axis=1)
wind_info

In [ ]:
years = range(2010, 2021)
gridded_data_speed = {}
gridded_data_x = {}
gridded_data_y = {}


resolution = 100
points_x, points_y = basemap(wind_info.longitude.values, wind_info.latitude.values)
x = np.linspace(np.min(points_x), np.max(points_x), resolution)
y = np.linspace(np.min(points_y), np.max(points_y), resolution)
X, Y = np.meshgrid(x, y)

for year in tqdm(years):
    # Initialize data
    year_data = wind_info[wind_info.year == year]
    
    points_x, points_y = basemap(year_data.longitude.values, year_data.latitude.values)
    
    # Interpolate wind speed and wind direction in grid
    z = griddata((points_x, points_y), year_data.wind_speed, (X, Y), method="linear", rescale=True, fill_value=0)
    u = griddata((points_x, points_y), year_data.u, (X, Y), method="linear", rescale=True, fill_value=0)
    v = griddata((points_x, points_y), year_data.v, (X, Y), method="linear", rescale=True, fill_value=0)

    gridded_data_speed.update({year : z})
    gridded_data_x.update({year : u})
    gridded_data_y.update({year : v})

In [ ]:
gridded_data_info = {"speed" : gridded_data_speed, 
                     "x" : gridded_data_x, 
                     "y" :gridded_data_y}

In [ ]:
year_test = 2014

fig, ax = plt.subplots(figsize = (20, 20))
basemap = viz_init(ax)

v = np.int32(np.linspace(
    0,
    np.quantile(np.nan_to_num(wind_info.wind_speed, 0), 0.98),
    10))

cnt = basemap.contourf(X, Y, gridded_data_info['speed'][year_test], v, cmap='jet', alpha=0.85)
#arr = basemap.quiver(X, Y, gridded_data_info['x'][year_test], gridded_data_info['y'][year_test], alpha=0.5)
plt.colorbar(cnt, ticks=v)
plt.show()